<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/evaluation/iss/RoBERTa_ISS_augmented_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Investigating text data augmentation using back translation for author profiling
- - - 
RoBERTa ON ISS DS EXPERIMENTS NOTEBOOK 
- - -
RoBERTa on Irony and Stereotype Spreaders Dataset augmented with DE backtranslation.
Code by M. Siino. 

## Importing modules.

In [ ]:
!pip install simpletransformers
#!pip install wandb==0.13.3 --upgrade
import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf
import pandas as pd

import torch

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#from google.colab import files
from io import open
from numpy.random import seed
import numpy as np
from pathlib import Path
from sklearn import svm
from simpletransformers.classification import ClassificationModel, ClassificationArgs
os.environ['TF_CUDNN_DETERMINISTIC']='true'
os.environ['TF_DETERMINISTIC_OPS']='true'

C:\Users\Domenico\.conda\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importing DS and extract in current working directory.

In [ ]:
urlTrainingSet = "https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-training-augmented-de.zip"
training_set = tf.keras.utils.get_file("pan22-author-profiling-training-2022-03-29-augmented-de.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

urlTestSet="https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-test-augmented-de.zip"
test_set = tf.keras.utils.get_file("pan22-author-profiling-test-2022-04-22-without_truth-augmented-de.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

In [ ]:
training_set_dir = os.path.join(os.path.dirname(training_set), 'pan22-author-profiling-training-2022-03-29-augmented-de')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan22-author-profiling-test-2022-04-22-without_truth-augmented-de')

!ls -A

'ls' is not recognized as an internal or external command,
operable program or batch file.


## Build folders hierarchy to use Keras folders preprocessing function.

In [ ]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_de'):
    os.makedirs('train_dir_de')

# Class labels directory.
if not os.path.exists('train_dir_de/0'):
    os.makedirs('train_dir_de/0')
if not os.path.exists('train_dir_de/1'):
    os.makedirs('train_dir_de/1')

# Make Py variables.
train_dir='train_dir_de'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_de'):
    os.makedirs('test_dir_de')

# Class labels directory.
if not os.path.exists('test_dir_de/0'):
    os.makedirs('test_dir_de/0')
if not os.path.exists('test_dir_de/1'):
    os.makedirs('test_dir_de/1')

# Make Py variables.
test_dir='test_dir_de'

!ls -A

'ls' is not recognized as an internal or external command,
operable program or batch file.


## Set language and directory paths.


In [ ]:
# Set en and es ground truth file path for train_dir. 
language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_en = truth_file_test_dir+'/'+'truth'+'.txt'

## Read truth.txt to organize training and test dataset folders.



In [ ]:
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]
    if label == 'I':
      label = '1'
    elif label == 'N':
      label = '0'

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_de/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]
    if label == 'I':
      label = '1'
    elif label == 'N':
      label = '0'

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_de/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

FileExistsError: ignored

## Generate full training set.



In [ ]:
# Generate full randomized training set.
batch_size=1

en_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir, 
    batch_size=batch_size,
    shuffle=False
    )

en_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir, 
    batch_size=batch_size,
    shuffle=False
    )

train_ds=en_train_ds.shuffle(500,seed=1, reshuffle_each_iteration=False)
test_ds=en_test_ds.shuffle(200,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

Found 420 files belonging to 2 classes.
Found 180 files belonging to 2 classes.


## Functions to pre-process source text. 

In [ ]:
# Preprocessing function to remove some noise due to the translation.
def clean_samples(input_data):
  tag_author_lang_en_removed = tf.strings.regex_replace(input_data,'', '')  
  tag_opening_documents = tf.strings.regex_replace(tag_author_lang_en_removed,'', '')
  tag_opening_cdata_removed = tf.strings.regex_replace(tag_opening_documents,'<\!\[CDATA\[', ' ')
  tag_closing_cdata_removed = tf.strings.regex_replace(tag_opening_cdata_removed,'\]\]>', ' >')
  tag_closing_documents = tf.strings.regex_replace(tag_closing_cdata_removed,'', '')
  output_data = tf.strings.regex_replace(tag_closing_documents,'', '')
  tag_author_lang_en_removed = tf.strings.regex_replace(output_data,'<author lang="en"', '')
  tag_author_lang_en_cased_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'<Author lang="en"', '\n\n')
  tag_opening_document_miscased = tf.strings.regex_replace(tag_author_lang_en_cased_removed,'<Document>', '<document>')
  output_data = tf.strings.regex_replace(tag_opening_document_miscased,'</Document>', '</document>')
  return output_data

## Function to convert DSs to Pandas Dataframe

In [ ]:
def preprocess_and_convert_ds(clean_samples):
  # Convert English dataset.
  train_df = [] # will contain text and label
  for element in train_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = clean_samples(authorDocument[0].numpy()).numpy().decode('UTF-8')
    train_df.append({
        'text':text,
        'label':label
    })
  train_df = pd.DataFrame(train_df)

  test_df = [] # will contain text and label
  for element in test_ds:
    authorDocument=element[0]
    label=int(element[1].numpy())
    #print(authorDocument[0])
    text = clean_samples(authorDocument[0].numpy()).numpy().decode('UTF-8')
    test_df.append({
        'text':text,
        'label':label
    })
  test_df = pd.DataFrame(test_df)

  return train_df, test_df


## Some parameters definition...

In [ ]:
# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

num_epochs_per_run = 10
num_runs = 5

Cuda available?  False


## Get the length of the longest sample in training set. Then adapt text.



In [ ]:
def preprocess_and_adapt_ts(training_set):
  # Set a large sequence length to find the longest sample in the training set.
  sequence_length = 30000
  vectorize_layer = TextVectorization(
      standardize=clean_samples,
      output_mode='int',
      output_sequence_length=sequence_length)

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]
    
    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    out=model(authorDocument)
    # Convert token list to numpy array.
    token_list = out.numpy()[0]
    token_list = np.trim_zeros(token_list,'b')
    if longest_sample_length < len(token_list):
      longest_sample_length = len(token_list)

  print("Length of the longest sample is:", longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  sequence_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=clean_samples,
      output_mode='int',
      output_sequence_length=sequence_length)

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  return vectorize_layer

## Vectorization




In [ ]:
print("\n\n* * * * VECTORIZATION STARTED * * * *")

# Preprocess training set to build a dictionary.
vectorize_layer = preprocess_and_adapt_ts(train_ds)

max_features=len(vectorize_layer.get_vocabulary()) + 1
print("Vocabulary size is:", max_features)



* * * * VECTORIZATION STARTED * * * *
Length of the longest sample is: 19539
Vocabulary size is: 229593


## Models definition and evaluation.




In [ ]:
print("\n\n***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******")
# Print a raw and a preprocessed sample.
for element in train_ds:
  authorDocument=element[0]
  label=element[1]
  
  print("Sample considered is: ", authorDocument[0])
  print("Preprocessed: ", str(clean_samples(authorDocument[0].numpy())))
  break

# # # - - - - - MODELS DEFINITION AND EVALUATION - - - - - # # #

model_args = ClassificationArgs(num_train_epochs=1, 
                                    no_save=True, 
                                    no_cache=True, 
                                    overwrite_output_dir=True)

model = ClassificationModel("roberta", 
                                'roberta-base', 
                                args = model_args, 
                                num_labels=2, 
                                use_cuda=cuda_available)

runs_accuracy = []

print("\n\n* * * * EVALUATION USING AS PREPROCESSING FUNCTION * * * *")

# Preprocess train and test set and convert to DFs.
train_df,test_df = preprocess_and_convert_ds(clean_samples)

for run in range(1,(num_runs+1)):
  epochs_accuracy=[]
  model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
  for epoch in range (0,num_epochs_per_run):
    print("\nEPOCH NUMBER: ", epoch)
    # train model
    print("\nNOW TRAIN THE MODEL.")
    model.train_model(train_df,show_running_loss=False)
    print("\nNOW EVALUATE THE TEST DF.")
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    # Results on test set.
    print(result)
    correct_predictions = result['tp']+result['tn']
    print("Correct predictions are: ",correct_predictions)
    total_predictions = result['tp']+result['tn']+result['fp']+result['fn']
    print("Total predictions are: ",total_predictions)
    accuracy = correct_predictions/total_predictions
    print("Accuracy on test set is:",accuracy,"\n\n")
    epochs_accuracy.append(accuracy)

  print(epochs_accuracy)
  runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies are:", runs_accuracy)
print("The median is:",runs_accuracy[2])

if (runs_accuracy[2]-runs_accuracy[0])>(runs_accuracy[4]-runs_accuracy[2]):
  max_range_from_median = runs_accuracy[2]-runs_accuracy[0]
else:
  max_range_from_median = runs_accuracy[4]-runs_accuracy[2]
final_result = str(runs_accuracy[2])+" +/- "+ str(max_range_from_median)

print("RoBERTa Accuracy Score on Test set -> ",final_result)



***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******
Sample considered is:  tf.Tensor(b' \n\t<document> pete buttigieg\xe2\x80\x99s community renewal visas plan - vox #url# </document>\n\t\t<document> #hashtag# #hashtag# mt #user#: one of the saddest effects of govt spying on muslims: #url# #url# </document>\n\t\t<document> #user# not that i thought much of him before but now i can officially write him off. just incredibly stupid. </document>\n\t\t<document> #user# #user# #user# it\xe2\x80\x99s not just other christians, plenty of conservative catholics have decided that since he doesn\xe2\x80\x99t agree with their worldview (and not just on covid) he is basically not legitimate. </document>\n\t\t<document> #user# baby steps but has anybody been fired before for not having body cams on? </document>\n\t\t<document> your illinois democrats everybody-all about the working people. #hashtag# rt #user#: wtf cc:#user# #url# </document>\n\t\t<document> reminde

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi



* * * * EVALUATION USING AS PREPROCESSING FUNCTION * * * *


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi


EPOCH NUMBER:  0

NOW TRAIN THE MODEL.


...

{'mcc': 0.5345224838248488, 'tp': 72, 'tn': 66, 'fp': 24, 'fn': 18, 'auroc': 0.8666666666666667, 'auprc': 0.8612216724024638, 'eval_loss': 1.043357714160305}
Correct predictions are:  138
Total predictions are:  180
Accuracy on test set is: 0.7666666666666667 



EPOCH NUMBER:  9

NOW TRAIN THE MODEL.


Epoch 1 of 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:56<00:00, 116.20s/it]
C:\Users\Domenico\.conda\envs\tf\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(



NOW EVALUATE THE TEST DF.


Running Evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:12<00:00,  1.79it/s]

{'mcc': 0.5471872549744599, 'tp': 74, 'tn': 65, 'fp': 25, 'fn': 16, 'auroc': 0.8708641975308642, 'auprc': 0.8678400181060617, 'eval_loss': 0.8743444395292065}
Correct predictions are:  139
Total predictions are:  180
Accuracy on test set is: 0.7722222222222223 


[0.6555555555555556, 0.7444444444444445, 0.7777777777777778, 0.7333333333333333, 0.7666666666666667, 0.7388888888888889, 0.7888888888888889, 0.7555555555555555, 0.7666666666666667, 0.7722222222222223]


 Over all runs maximum accuracies are: [0.7777777777777778, 0.7888888888888889, 0.8055555555555556, 0.8166666666666667, 0.8166666666666667]
The median is: 0.8055555555555556
RoBERTa Accuracy Score on Test set ->  0.8055555555555556 +/- 0.02777777777777779
